## fine tuning using data from retail

In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from sentence_transformers import losses, util
from sentence_transformers import LoggingHandler, SentenceTransformer, evaluation
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import csv
import os
from zipfile import ZipFile
import random

In [2]:
#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
logger = logging.getLogger(__name__)
#### /print debug information to stdout

In [3]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch_device = 'cpu'

model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1',
                           device=torch_device)
num_epochs = 5
train_batch_size = 64

2022-11-15 11:02:58 - Load pretrained SentenceTransformer: multi-qa-MiniLM-L6-cos-v1


In [4]:
#As distance metric, we use cosine distance (cosine_distance = 1-cosine_similarity)
distance_metric = losses.SiameseDistanceMetric.COSINE_DISTANCE

#Negative pairs should have a distance of at least 0.80
margin = 0.5

dataset_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 
                            'data/retail_train/')

model_save_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 
                               'data/output/training_retailConstrativeLoss-')\
+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


os.makedirs(model_save_path, exist_ok=True)

In [5]:
######### Read train data  ##########
# Read train data
train_samples = []
with open(os.path.join(dataset_path, "classification/train_pairs.tsv"), encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:       
        sample = InputExample(texts=[row['product1'], row['product2']], label=int(row['is_duplicate']))
        train_samples.append(sample)

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.OnlineContrastiveLoss(model=model, distance_metric=distance_metric, margin=margin)

In [6]:
################### Development  Evaluators ##################
# We add 3 evaluators, that evaluate the model on Duplicate Products pair classification,
# Duplicate Products Mining, and Duplicate Products Information Retrieval
evaluators = []

###### Classification ######
# Given (product1, product2), is this a duplicate or not?
# The evaluator will compute the embeddings for both products and then compute
# a cosine similarity. If the similarity is above a threshold, we have a duplicate.
dev_sentences1 = []
dev_sentences2 = []
dev_labels = []
with open(os.path.join(dataset_path, "classification/dev_pairs.tsv"), encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        dev_sentences1.append(row['product1'])
        dev_sentences2.append(row['product2'])
        dev_labels.append(int(row['is_duplicate']))


binary_acc_evaluator = evaluation.BinaryClassificationEvaluator(dev_sentences1, 
                                                                dev_sentences2, 
                                                                dev_labels)
evaluators.append(binary_acc_evaluator)

In [7]:
###### Duplicate Products Mining ######
# Given a large corpus of products, identify all duplicates in that corpus.

max_dev_samples = 10000
dev_sentences = {}
dev_duplicates = []
with open(os.path.join(dataset_path, "duplicate-mining/dev_corpus.tsv"), encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        dev_sentences[row['qid']] = row['product']

        if len(dev_sentences) >= max_dev_samples:
            break

with open(os.path.join(dataset_path, "duplicate-mining/dev_duplicates.tsv"), encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        if row['qid1'] in dev_sentences and row['qid2'] in dev_sentences:
            dev_duplicates.append([row['qid1'], row['qid2']])

In [8]:
# The ParaphraseMiningEvaluator computes the cosine similarity between all sentences and
# extracts a list with the pairs that have the highest similarity. Given the duplicate
# information in dev_duplicates, it then computes and F1 score how well our duplicate mining worked
paraphrase_mining_evaluator = evaluation.ParaphraseMiningEvaluator(dev_sentences, 
                                                                   dev_duplicates, name='dev')
evaluators.append(paraphrase_mining_evaluator)

In [9]:
###### Duplicate Products Information Retrieval ######
# Given a product and a large corpus of thousands products, 
# find the most relevant (i.e. duplicate) product in that corpus.

# For faster processing, we limit the development corpus to only 10,000 sentences.
max_corpus_size = 10000

ir_queries = {}             #Our queries (qid => product)
ir_needed_qids = set()      #QIDs we need in the corpus
ir_corpus = {}              #Our corpus (qid => product)
ir_relevant_docs = {}       #Mapping of relevant documents for a given query 
                            #(qid => set([relevant_product_ids])

with open(os.path.join(dataset_path, 'information-retrieval/dev-queries.tsv'), encoding='utf8') as fIn:
    next(fIn) #Skip header
    for line in fIn:
        qid, query, duplicate_ids = line.strip().split('\t')
        duplicate_ids = duplicate_ids.split(',')
        ir_queries[qid] = query
        ir_relevant_docs[qid] = set(duplicate_ids)

        for qid in duplicate_ids:
            ir_needed_qids.add(qid)

In [10]:
# First get all needed relevant documents 
#(i.e., we must ensure, that the relevant products are actually in the corpus
distraction_products = {}
with open(os.path.join(dataset_path, 'information-retrieval/corpus.tsv'), encoding='utf8') as fIn:
    next(fIn) #Skip header
    for line in fIn:
        qid, product = line.strip().split('\t')
        if qid in ir_needed_qids:
            ir_corpus[qid] = product
        else:
            distraction_products[qid] = product

# Now, also add some irrelevant products to fill our corpus
other_qid_list = list(distraction_products.keys())
random.shuffle(other_qid_list)

for qid in other_qid_list[0:max(0, max_corpus_size-len(ir_corpus))]:
    ir_corpus[qid] = distraction_products[qid]

In [11]:
#Given queries, a corpus and a mapping with relevant documents, the InformationRetrievalEvaluator computes different IR
# metrices. For our use case MRR@k and Accuracy@k are relevant.
ir_evaluator = evaluation.InformationRetrievalEvaluator(ir_queries, ir_corpus, ir_relevant_docs)

evaluators.append(ir_evaluator)

In [12]:
# Create a SequentialEvaluator. This SequentialEvaluator runs all three evaluators in a sequential order.
# We optimize the model with respect to the score from the last evaluator (scores[-1])
seq_evaluator = evaluation.SequentialEvaluator(evaluators, 
                                               main_score_function=lambda scores: scores[-1])


logger.info("Evaluate model without training")
with torch.no_grad():
    seq_evaluator(model, epoch=0, steps=0, output_path=model_save_path)

2022-11-15 11:02:59 - Evaluate model without training
2022-11-15 11:02:59 - Binary Accuracy Evaluation of the model on  dataset in epoch 0 after 0 steps:
2022-11-15 11:03:06 - Accuracy with Cosine-Similarity:           79.27	(Threshold: 0.8767)
2022-11-15 11:03:06 - F1 with Cosine-Similarity:                 55.65	(Threshold: 0.7286)
2022-11-15 11:03:06 - Precision with Cosine-Similarity:          46.94
2022-11-15 11:03:06 - Recall with Cosine-Similarity:             68.32
2022-11-15 11:03:06 - Average Precision with Cosine-Similarity:  47.39

2022-11-15 11:03:06 - Accuracy with Manhattan-Distance:           79.27	(Threshold: 7.6162)
2022-11-15 11:03:06 - F1 with Manhattan-Distance:                 56.22	(Threshold: 11.4881)
2022-11-15 11:03:06 - Precision with Manhattan-Distance:          47.30
2022-11-15 11:03:06 - Recall with Manhattan-Distance:             69.31
2022-11-15 11:03:06 - Average Precision with Manhattan-Distance:  47.65

2022-11-15 11:03:06 - Accuracy with Euclidean-Di

In [13]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=seq_evaluator,
          epochs=num_epochs,
          warmup_steps=1000,
          output_path=model_save_path
         )

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/100 [00:00<?, ?it/s]

2022-11-15 11:03:29 - Binary Accuracy Evaluation of the model on  dataset after epoch 0:
2022-11-15 11:03:29 - Accuracy with Cosine-Similarity:           80.41	(Threshold: 0.8461)
2022-11-15 11:03:29 - F1 with Cosine-Similarity:                 57.47	(Threshold: 0.6871)
2022-11-15 11:03:29 - Precision with Cosine-Similarity:          46.88
2022-11-15 11:03:29 - Recall with Cosine-Similarity:             74.26
2022-11-15 11:03:29 - Average Precision with Cosine-Similarity:  49.63

2022-11-15 11:03:29 - Accuracy with Manhattan-Distance:           79.95	(Threshold: 8.4054)
2022-11-15 11:03:29 - F1 with Manhattan-Distance:                 57.37	(Threshold: 12.0156)
2022-11-15 11:03:29 - Precision with Manhattan-Distance:          48.00
2022-11-15 11:03:29 - Recall with Manhattan-Distance:             71.29
2022-11-15 11:03:29 - Average Precision with Manhattan-Distance:  49.74

2022-11-15 11:03:29 - Accuracy with Euclidean-Distance:           80.41	(Threshold: 0.5548)
2022-11-15 11:03:29 -

Iteration:   0%|          | 0/100 [00:00<?, ?it/s]

2022-11-15 11:03:53 - Binary Accuracy Evaluation of the model on  dataset after epoch 1:
2022-11-15 11:03:53 - Accuracy with Cosine-Similarity:           81.09	(Threshold: 0.8275)
2022-11-15 11:03:53 - F1 with Cosine-Similarity:                 61.54	(Threshold: 0.7002)
2022-11-15 11:03:53 - Precision with Cosine-Similarity:          52.05
2022-11-15 11:03:53 - Recall with Cosine-Similarity:             75.25
2022-11-15 11:03:53 - Average Precision with Cosine-Similarity:  53.00

2022-11-15 11:03:53 - Accuracy with Manhattan-Distance:           80.64	(Threshold: 8.8616)
2022-11-15 11:03:53 - F1 with Manhattan-Distance:                 62.35	(Threshold: 12.0719)
2022-11-15 11:03:53 - Precision with Manhattan-Distance:          52.74
2022-11-15 11:03:53 - Recall with Manhattan-Distance:             76.24
2022-11-15 11:03:53 - Average Precision with Manhattan-Distance:  53.10

2022-11-15 11:03:53 - Accuracy with Euclidean-Distance:           81.09	(Threshold: 0.5874)
2022-11-15 11:03:53 -

...

2022-11-15 11:04:17 - Binary Accuracy Evaluation of the model on  dataset after epoch 2:
2022-11-15 11:04:17 - Accuracy with Cosine-Similarity:           82.69	(Threshold: 0.7672)
2022-11-15 11:04:17 - F1 with Cosine-Similarity:                 64.66	(Threshold: 0.7419)
2022-11-15 11:04:17 - Precision with Cosine-Similarity:          57.25
2022-11-15 11:04:17 - Recall with Cosine-Similarity:             74.26
2022-11-15 11:04:17 - Average Precision with Cosine-Similarity:  55.87

2022-11-15 11:04:17 - Accuracy with Manhattan-Distance:           82.46	(Threshold: 10.4675)
2022-11-15 11:04:17 - F1 with Manhattan-Distance:                 64.66	(Threshold: 11.2316)
2022-11-15 11:04:17 - Precision with Manhattan-Distance:          57.25
2022-11-15 11:04:17 - Recall with Manhattan-Distance:             74.26
2022-11-15 11:04:17 - Average Precision with Manhattan-Distance:  56.11

2022-11-15 11:04:17 - Accuracy with Euclidean-Distance:           82.69	(Threshold: 0.6824)
2022-11-15 11:04:17 

Iteration:   0%|          | 0/100 [00:00<?, ?it/s]

2022-11-15 11:04:42 - Binary Accuracy Evaluation of the model on  dataset after epoch 3:
2022-11-15 11:04:43 - Accuracy with Cosine-Similarity:           84.51	(Threshold: 0.7979)
2022-11-15 11:04:43 - F1 with Cosine-Similarity:                 68.22	(Threshold: 0.7903)
2022-11-15 11:04:43 - Precision with Cosine-Similarity:          64.60
2022-11-15 11:04:43 - Recall with Cosine-Similarity:             72.28
2022-11-15 11:04:43 - Average Precision with Cosine-Similarity:  58.11

2022-11-15 11:04:43 - Accuracy with Manhattan-Distance:           84.51	(Threshold: 9.9875)
2022-11-15 11:04:43 - F1 with Manhattan-Distance:                 67.92	(Threshold: 9.9875)
2022-11-15 11:04:43 - Precision with Manhattan-Distance:          64.86
2022-11-15 11:04:43 - Recall with Manhattan-Distance:             71.29
2022-11-15 11:04:43 - Average Precision with Manhattan-Distance:  58.36

2022-11-15 11:04:43 - Accuracy with Euclidean-Distance:           84.51	(Threshold: 0.6358)
2022-11-15 11:04:43 - 

Iteration:   0%|          | 0/100 [00:00<?, ?it/s]

2022-11-15 11:05:07 - Binary Accuracy Evaluation of the model on  dataset after epoch 4:
2022-11-15 11:05:07 - Accuracy with Cosine-Similarity:           85.42	(Threshold: 0.8512)
2022-11-15 11:05:07 - F1 with Cosine-Similarity:                 69.41	(Threshold: 0.7801)
2022-11-15 11:05:07 - Precision with Cosine-Similarity:          64.41
2022-11-15 11:05:07 - Recall with Cosine-Similarity:             75.25
2022-11-15 11:05:07 - Average Precision with Cosine-Similarity:  60.67

2022-11-15 11:05:07 - Accuracy with Manhattan-Distance:           85.19	(Threshold: 8.4650)
2022-11-15 11:05:07 - F1 with Manhattan-Distance:                 68.47	(Threshold: 10.3277)
2022-11-15 11:05:07 - Precision with Manhattan-Distance:          62.81
2022-11-15 11:05:07 - Recall with Manhattan-Distance:             75.25
2022-11-15 11:05:07 - Average Precision with Manhattan-Distance:  60.63

2022-11-15 11:05:07 - Accuracy with Euclidean-Distance:           85.42	(Threshold: 0.5454)
2022-11-15 11:05:07 -